In [1]:
import bk.load
import numpy as np
import matplotlib.pyplot as plt

import bk.compute
import neuroseries as nts

import cv2

In [2]:
def CountTTL(TTL):
    #Return the number TTL and the index where the last one starts
    
    TTL = list(map(int,TTL))
    diff_TTL = np.diff(TTL)
    
    t_start = np.where(diff_TTL == 1)

    return(len(t_start[0]),t_start[0][-1])

def loadDigitalin(path_digitalin):
    digital_word = np.fromfile(path_digitalin, "uint16")
    data = (digital_word & 2 ** 0) > 0
    return data


def CountFramesSlow(path):
    cap = cv2.VideoCapture(path)
    
    count = 0
    while True:
        (grabbed, frames) = cap.read()
        if not grabbed:
            break
        count += 1
    return count

def CountFramesFast(path):

    cap = cv2.VideoCapture(path)
    return(int(cap.get(cv2.CAP_PROP_FRAME_COUNT)))

In [3]:
path = "/mnt/electrophy/Billel/BK-Dataset/Tests/Rat00/Rat00-20220516/Rat00_220516_112817/"
os.chdir(path)
ls = os.listdir(path)

In [4]:
dat = os.path.join(path,'digitalin.dat')
# video = [v for v in ls if v.endswith('mp4')][0]
# video = os.path.join(path,video)

In [5]:
ttl = loadDigitalin(dat)
CountTTL(ttl)

(238883, 159285455)

In [25]:
ttl = loadDigitalin(dat)
ttl_intervals = bk.compute.TTL_to_intervals(ttl)
inter = nts.IntervalSet(ttl_intervals[0],ttl_intervals[1],time_units="s")
inter.merge_close_intervals(1500)

ttl_mid = np.mean(inter,axis = 1).values/1_000_000
small_frames = ttl_mid[inter.duration()<28500]

inter

In [99]:
CountFramesFast(video)

70177

In [65]:
%matplotlib qt
t = np.arange(0,len(ttl)/20000,1/20000)

plt.plot(t,ttl)
for i in small_frames:
    plt.axvline(i,color = 'red')

In [37]:
np.unique(inter.duration())

array([  100,   300,  1800,  2000,  2300,  6850,  9200, 10250, 11900,
       16900, 19050, 20000, 20600, 22400, 26950, 27300, 27450, 29299,
       29300, 29301, 29349, 29350, 29351])

# ELE DATA

In [1]:
import bk.load
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

import bk.compute
import neuroseries as nts

import cv2

In [14]:
def CountTTL(TTL):
    #Return the number TTL and the index where the last one starts
    
    TTL = list(map(int,TTL))
    diff_TTL = np.diff(TTL)
    
    t_start = np.where(diff_TTL == 1)

    return(len(t_start[0]),t_start[0][-1])

def loadDigitalin(path_digitalin):
    digital_word = np.fromfile(path_digitalin, "uint16")
    data = (digital_word & 2 ** 0) > 0
    return data


def CountFramesSlow(path):
    cap = cv2.VideoCapture(path)
    
    count = 0
    while True:
        (grabbed, frames) = cap.read()
        if not grabbed:
            break
        count += 1
    return count

def CountFramesFast(path):

    cap = cv2.VideoCapture(path)
    return(int(cap.get(cv2.CAP_PROP_FRAME_COUNT)))

In [15]:
paths = pd.read_csv('/mnt/electrophy/Eleonore/Behavior/behavpaths.csv')

In [45]:
df = pd.DataFrame()
for path in paths['path']:
    os.chdir(path)
    ls = os.listdir(path)
    session = path.split('/')[-1]
    dat_path = os.path.join(path,'digitalin.dat')
    video = [v for v in ls if v.endswith('mp4')][0]

    ttl = loadDigitalin(dat_path)
    ttl_intervals = bk.compute.TTL_to_intervals(ttl)
    inter = nts.IntervalSet(ttl_intervals[0],ttl_intervals[1],time_units="s")
    inter.merge_close_intervals(1500)


    n_ttl = CountTTL(ttl)[0]
    n_frames = CountFramesFast(video)
    

    d = pd.DataFrame([[session, n_ttl,len(inter),n_frames,n_ttl-n_frames]],columns=('Session','TTL','Removing_Small_TTLs','Frames','delta'))
    df = pd.concat((df,d))
    print(f'Session : {session}')
    print(f'TTL : {n_ttl}')
    print(f'Frames : {n_frames} \n')  
    

Session : Rat58_211007_091454
TTL : 16131
Frames : 16130 

Session : Rat58_211007_132848
TTL : 10406
Frames : 10405 

Session : Rat58_211007_143542
TTL : 8747
Frames : 8746 

Session : Rat59_211008_094030
TTL : 15524
Frames : 15523 

Session : Rat59_211008_134504
TTL : 15224
Frames : 15223 

Session : Rat59_211008_144602
TTL : 15340
Frames : 15339 

Session : Rat61_211027_103118
TTL : 15797
Frames : 15796 

Session : Rat61_211027_143348
TTL : 15213
Frames : 15211 

Session : Rat61_211027_153431
TTL : 15238
Frames : 15237 

Session : Rat62_211028_103112
TTL : 15321
Frames : 15320 

Session : Rat62_211028_143534
TTL : 16273
Frames : 16272 

Session : Rat62_211028_153438
TTL : 15227
Frames : 15226 

Session : Rat63_211029_103016
TTL : 15335
Frames : 15334 

Session : Rat63_211029_143849
TTL : 15506
Frames : 15505 

Session : Rat63_211029_153554
TTL : 15353
Frames : 15352 

Session : Rat64_211117_103119
TTL : 15939
Frames : 15938 

Session : Rat64_211117_142937
TTL : 16010
Frames : 16009 


In [42]:
os.chdir('/home/billel/Documents')

In [46]:
df.to_csv('test.csv')